In [1]:
import re
from typing import Tuple, Dict

def pii_filter(text: str) -> Tuple[str, Dict[str, str]]:
    """
    Replaces Email IDs, Phone Numbers, and SSNs with placeholders.
    Returns filtered_text and mapping for restoration.
    """

    patterns = {
        "EMAIL": r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}",
        "PHONE": r"\b(?:\+?1[-.\s]?)?(?:\(?\d{3}\)?[-.\s]?|\d{3}[-.\s]?)\d{3}[-.\s]?\d{4}\b",
        "SSN":   r"\b\d{3}-\d{2}-\d{4}\b"
    }

    mapping = {}
    counter = {"EMAIL": 0, "PHONE": 0, "SSN": 0}

    def replace(match, pii_type):
        counter[pii_type] += 1
        key = f"[{pii_type}_{counter[pii_type]}]"
        mapping[key] = match.group(0)
        return key

    # Apply patterns
    for pii_type, pattern in patterns.items():
        text = re.sub(pattern, lambda m: replace(m, pii_type), text)

    return text, mapping


def pii_restore(text: str, mapping: Dict[str, str]) -> str:
    """
    Restores placeholders in text back to original PII values.
    """
    for placeholder, original in mapping.items():
        text = text.replace(placeholder, original)
    return text


# ---------------- Example ----------------
sample_text = """
Contact John via email john.doe@example.com or phone (123) 456-7890.
His SSN is 123-45-6789. Another email: jane_smith@test.org and phone 987-654-3210.
"""

# Filter PII
filtered_text, pii_map = pii_filter(sample_text)
print("Filtered Text:\n", filtered_text)
print("\nMapping:\n", pii_map)

# Restore PII
restored_text = pii_restore(filtered_text, pii_map)
print("\nRestored Text:\n", restored_text)


Filtered Text:
 
Contact John via email [EMAIL_1] or phone ([PHONE_1].
His SSN is [SSN_1]. Another email: [EMAIL_2] and phone [PHONE_2].


Mapping:
 {'[EMAIL_1]': 'john.doe@example.com', '[EMAIL_2]': 'jane_smith@test.org', '[PHONE_1]': '123) 456-7890', '[PHONE_2]': '987-654-3210', '[SSN_1]': '123-45-6789'}

Restored Text:
 
Contact John via email john.doe@example.com or phone (123) 456-7890.
His SSN is 123-45-6789. Another email: jane_smith@test.org and phone 987-654-3210.

